In [1]:
!pip install otter-grader

In [2]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

ValueError: Tests directory ./tests does not exist

# CPSC 330 - Applied Machine Learning

## Homework 7: Word embeddings and topic modeling
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [3]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Instructions
rubric={points}

You will earn points for following these instructions and successfully submitting your work on Gradescope.  

### Group wotk instructions

**You may work with a partner on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2.
  
- Use group work as an opportunity to collaborate and learn new things from each other.
- Be respectful to each other and make sure you understand all the concepts in the assignment well.
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline.
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).
- If you would like to use late tokens for the homework, all group members must have the necessary late tokens available. Please note that the late tokens will be counted for all members of the group.   


### General submission instructions

- Please **read carefully
[Use of Generative AI policy](https://ubc-cs.github.io/cpsc330-2025W1/syllabus.html#use-of-generative-ai-in-the-course)** before starting the homework assignment.
- **Run all cells before submitting:** Go to `Kernel -> Restart Kernel and Clear All Outputs`, then select `Run -> Run All Cells`. This ensures your notebook runs cleanly from start to finish without errors.
  
- **Submit your files on Gradescope.**  
   - Upload only your `.ipynb` file **with outputs displayed** and any required output files.
     
   - Do **not** submit other files from your repository.  
   - If you need help, see the [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/).  
- **Check that outputs render properly.**  
   - Make sure all plots and outputs appear in your submission.
     
   - If your `.ipynb` file is too large and doesn't render on Gradescope, also upload a PDF or HTML version so the TAs can view your work.  
- **Keep execution order clean.**  
   - Execution numbers must start at "1" and increase in order.
     
   - Notebooks without visible outputs may not be graded.  
   - Out-of-order or missing execution numbers may result in mark deductions.  
- **Follow course submission guidelines:** Review the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html) for detailed guidance on completing and submitting assignments.
   
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl.

A number of pre-trained word embeddings are available out there. Some popular ones are:

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf)
    * published by Stanford University
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html)
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below.

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [4]:
!pip install gensim

In [5]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [6]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [7]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model.

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings.

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [8]:
print('The 4 words of my choice are: light, plant, laptop, phone')
words = ['light', 'plant', 'laptop', 'phone']

The 4 words of my choice are: light, plant, laptop, phone


In [9]:
for word in words:
  print(word)
  for similar, score in glove_wiki_vectors.most_similar(word, topn=10):
        print(f"  {similar}")

light
  dark
  bright
  lights
  air
  blue
  visible
  color
  sky
  heavy
  water
plant
  plants
  factory
  farm
  facility
  production
  produce
  processing
  fertilizer
  waste
  factories
laptop
  laptops
  computers
  phones
  portable
  desktop
  ipod
  computer
  handheld
  pc
  cellphones
phone
  telephone
  cellphone
  phones
  mobile
  mail
  internet
  messages
  wireless
  telephones
  cellular


In [10]:
...

Ellipsis

In [11]:
...

Ellipsis

In [12]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [13]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [14]:
for word1, word2 in word_pairs:
    sim = glove_wiki_vectors.similarity(word1, word2)
    print(f"Similarity({word1}, {word2}) = {sim:.4f}")

Similarity(coast, shore) = 0.7000
Similarity(clothes, closet) = 0.5463
Similarity(old, new) = 0.6432
Similarity(smart, intelligent) = 0.7553
Similarity(dog, cat) = 0.8798
Similarity(tree, lawyer) = 0.0767


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not.
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True.

In [15]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [16]:
for word in test_words:
    print(word, word in glove_wiki_vectors)

covididiot False
fomo False
frenemies True
anthropause False
photobomb False
selfie False
pxg False
pacg False
cct True
escc True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data.

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this.
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models.

In [17]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [18]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [19]:
glove_wiki_vectors.similarity("aboriginal", "success")

np.float32(0.14283238)

In [20]:
glove_wiki_vectors.similarity("white", "success")

np.float32(0.35182396)

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [21]:
glove_wiki_vectors.similarity("indian", "violence")


np.float32(0.36956844)

In [22]:
glove_wiki_vectors.similarity("canadian", "violence")

np.float32(0.18276742)

In [23]:
analogy("white", "rich", "coloured")

white : rich :: coloured : ?


,Analogy word,Score
0,richly,0.565491
1,mineral,0.564086
2,richer,0.544013
3,abundant,0.528304
4,fertile,0.524897
5,diverse,0.508858
6,textures,0.498944
7,textured,0.497468
8,fabulously,0.488432
9,alluvial,0.488141


In [24]:
analogy("men", "competent", "blondes")

men : competent :: blondes : ?


,Analogy word,Score
0,unadventurous,0.617484
1,fair-minded,0.587928
2,spineless,0.549455
3,cool-headed,0.547146
4,self-assured,0.543766
5,hardheaded,0.541290
6,venturesome,0.533579
7,incorruptible,0.533527
8,unclassifiable,0.531733
9,maladroit,0.531262


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

1. The only one that wasn't concerning was the third example of "white, rich, and coloured". The first two examples were worrisome because the score for the similarity between violence and Indians was double the score for the similarity between Canadians and violence. The last example only had a few concerning similar words, such as spineless and unadventurous, however there are positive adjectives for them as well.

2. When used by LLMs such as ChatGPT, they may perpetuate these stereotypes and biases to a user who may not have the ability to understand that stereotypes can be false.
For example, younger kids have access to ChatGPT before having a full picture of how the world really is, so they may trust that everything that ChatGPT says is true. Given this, if, for example, that user asks ChatGPT about coloured people or about white people, ChatGPT may associate the word violence to coloured people more than they would for white people, and the kid may then also have a stereotype in their head that coloured people are more violent because they have no one supervising the information they learn from ChatGPT(using violence as an example since I used that word in 1.4, but violence can be substituted with another negative stereotype).

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling

The goal of topic modeling is discovering high-level themes in a large collection of texts.

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model.

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics.

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories.

In [25]:
from sklearn.datasets import fetch_20newsgroups

In [26]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [27]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling.

**Your tasks:**

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md.

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include:
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM".
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing.

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy.

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps.

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook.



In [28]:
!pip install -U spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 47.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [29]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [30]:
#pulled from lecture 17, and added a few more if statements above such as the url, num, and punctuation
import re # I didn't know what library to use for regex, so I asked chatgpt what library I can use for it, and it told me to import re. I then looked up documentation for re
def regex(token):
  token = re.sub(r"\n+", " ", token)
  token = re.sub(r"[^A-Za-z0-9\s]", " ", token)
  token = re.sub(r"\s+", " ", token)
  return token.strip()

def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    clean_text = []

    for token in doc:
      if token.lower_ == "n't":
            clean_tokens.append("not")
            continue
      if token.is_space or token.is_punct:
            continue
      if (token.like_url):
            clean_text.append("URL")
            continue
      if (token.like_email):
            clean_text.append("EMAIL")
            continue
      if (token.like_num):
            clean_text.append("NUM")
            continue
      if (
          token.is_stop == False  # Check if it's not a stopword
          and len(token) > min_token_len  # Check if the word meets minimum threshold
          and token.pos_ not in irrelevant_pos):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [31]:
df["clean"] = df["text"].apply(regex)
df["preprocessed"] = [preprocess(doc) for doc in nlp.pipe(df["clean"])]
df.to_csv("preprocessed.csv", index=False)

In [32]:
...

Ellipsis

In [33]:
df.iloc[2:6]

,text,target,target_name,clean,preprocessed
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,Actuallay I don t but on the other hand I don ...,actuallay don hand don support idea have NUM n...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,The following problem is really bugging me and...,follow problem bug appreciate help create NUM ...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,This is the latest from UPI Foreign Ministry s...,late upi foreign ministry spokesman ferhat ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,Hi I d like to subscribe to Leadership Magazin...,like subscribe leadership magazine wonder NUM ...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 4

I picked 8 because we are using 8 out of the 20 categories

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

preprocessed_csv = pd.read_csv("preprocessed.csv")
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["preprocessed"])

n_topics = 8
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42
)
document_topics = lda.fit_transform(X)

In [35]:
document_topics

array([[2.15818247e-03, 2.15922312e-03, 2.15730090e-03, ...,
        8.25029862e-01, 2.15761149e-03, 1.62020166e-01],
       [8.72354922e-01, 1.50832848e-03, 1.50812977e-03, ...,
        1.50832514e-03, 1.50785812e-03, 1.50986958e-03],
       [4.27790859e-01, 1.92662859e-03, 1.92553999e-03, ...,
        1.92558809e-03, 1.92608033e-03, 5.60654000e-01],
       ...,
       [2.40614917e-03, 2.40931943e-03, 2.40633544e-03, ...,
        8.74530265e-01, 2.40527021e-03, 1.11025631e-01],
       [6.03005589e-01, 3.20663185e-03, 3.20583550e-03, ...,
        3.20875137e-03, 2.39977360e-01, 3.20977255e-03],
       [3.78066028e-04, 3.78102418e-04, 3.78047146e-04, ...,
        9.26208312e-01, 7.15234685e-02, 3.78182163e-04]])

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class).

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 5

_Type your answer here, replacing this text._

In [36]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())
feature_names

array(['000000e5', '00000ee5', '000010af', ..., 'zyxel', 'zzzzzz',
       'zzzzzzt'], dtype=object)

In [37]:
!pip install mglearn

In [38]:
import mglearn

mglearn.tools.print_topics(
    topics=range(8),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
num           num           israel        num           right         
window        god           jews          game          militia       
image         believe       israeli       team          state         
use           jesus         jewish        play          people        
file          people        arab          season        team          
program       know          state         win           num           
display       think         people        year          think         
widget        say           right         period        define        
color         church        war           player        know          
run           question      country       league        don           


topic 5       topic 6       topic 7       
--------      --------      --------      
num           num           num           
gun           fil

Topic 0 would be UI/grapics on a PC. Topic 1 would be sermons (talking about religion, asking questions about it, etc.), topic 2 would be Middle Eastern Politics, topic 3 would be competitions, topic 4 would be military, topic 5 would be government & law, topic 6 would be education (basing it off edu, however the outliers would be turkish and armenian),education topic 7 would be future events (has time/arrival words)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`.

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [39]:
topic_labels = {
    0: "PC UI/grapics",
    1: "Sermons",
    2: "Middle Eastern Politics",
    3: "Competitions",
    4: "Military",
    5: "Government & Law",
    6: "Education",
    7: "Future Events"
}
assigned_topic1 = np.argmax(document_topics[0])
assigned_topic2 = np.argmax(document_topics[1])
assigned_topic3 = np.argmax(document_topics[2])
assigned_topic4 = np.argmax(document_topics[3])
assigned_topic5 = np.argmax(document_topics[4])
label1 = topic_labels[assigned_topic1]
label2 = topic_labels[assigned_topic2]
label3 = topic_labels[assigned_topic3]
label4 = topic_labels[assigned_topic4]
label5 = topic_labels[assigned_topic5]
print(label1)
print(label2)
print(label3)
print(label4)
print(label5)

Government & Law
PC UI/grapics
Future Events
PC UI/grapics
Government & Law


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems.
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. Content-based filtering recommends items to a user based on similarity distances between new items/contents and previous items/contents that the user has engaged with. For example, if a user constantly watches action movies, the system will suggest more action movies. Using Netflix as an example, the system may use the genre tags for each movie (ex. titanic may have the genre tag of romance and drama) as features and compare those features with the features of previous items to see if they match (ex. comparing the genre of titanic to the genre of a movie I watched previously like James Bond would give me a low similarity score, so it would not recommend the titanic).

2. Filter bubbles/echo chambers (they would get recommendations that strongly echo the previous choices or media they have engaged with. This is especially harmful in social media when people only view content that aligns with their political views and they get no exposure to other opinions, thus limiting their worldview). The second negative consequence is popularity bias, where the same super popular content or products would constantly get recommended to users, while those who are just starting their social media accounts or just opening their businesses may have a hard time getting that initial exposure.

3. This is based off my understanding from the Piazza post. For real world tasks, we need complex models that would be very computationally expensive to build from scratch (ex. using chatbots in another language). Instead, we can use models that have already been trained on previously large datasets and reuse it to fit the task we want. In the context of NLPs, examples of this would be the Google News pre-trained embeddings, where they already have a large vocabulary/phrase bank, and we don't have to retrain the model, we can simply just plug in a word and it can give us similar words because it has been pre-trained. Another exmaple would be using sentence embeddings instead of BoW representation, giving us a better result because it was already pre-trained (as opposed to us training a model from scratch using BoW). Essentially, you are transferring the learning a model has gotten from the initial bulk pre-training and applying that learning into your own adaptation of a situation.

<!-- END QUESTION -->

<br><br><br><br>

Before submitting your assignment, please make sure you have followed all the instructions in the Submission Instructions section at the top.

Here is a quick checklist before submitting:

- [ ] Restart kernel, clear outputs, and run all cells from top to bottom.  
- [ ] `.ipynb` file runs without errors and contains all outputs.  
- [ ] Only `.ipynb` and required output files are uploaded (no extra files).  
- [ ] Execution numbers start at **1** and are in order.  
- [ ] If `.ipynb` is too large and doesn't render on Gradescope, also upload a PDF/HTML version.  
- [ ] Reviewed the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html).  

![](https://github.com/divagandhi/hw7/blob/main/img/eva-well-done.png?raw=1)